In [3]:
import pandas as pd
compare_df = pd.read_csv('smilarity_result.csv')
compare_df[compare_df['Description choose columns'] > 120]#[['Question','Description choose columns']]

,Question,SQL,Expect Columns,Columns-token Min threshold,Columns-token choose columns,Description Min threshold,Description choose columns
12,What is the average number of app opens each m...,"SELECT month, year, AVG(cnt) FROM\n (SELECT u...","['user_pseudo_id', 'event_name', 'event_timest...",0.301989,144,0.052267,128
13,How many times each user open the app?,"SELECT user_pseudo_id, COUNT(event_name) FROM ...","['user_pseudo_id', 'event_name']",0.393950,91,0.062655,132
17,How many users have opened the app more than o...,SELECT COUNT(DISTINCT user_pseudo_id) FROM\n ...,"['event_date', 'user_pseudo_id', 'event_name']",0.334538,117,0.086362,136
18,How many users have opened the app more than o...,SELECT COUNT(DISTINCT user_pseudo_id) FROM\n ...,"['event_date', 'user_pseudo_id', 'event_name']",0.334538,117,0.093939,121
19,How many users have opened the app everyday at...,SELECT COUNT(DISTINCT user_pseudo_id) FROM\n ...,"['event_date', 'user_pseudo_id', 'event_name']",0.277931,131,0.063201,141
20,How many users have opened the app everyday at...,SELECT COUNT(DISTINCT user_pseudo_id) FROM\n ...,"['event_date', 'user_pseudo_id', 'event_name']",0.277931,131,0.064723,136
21,How many users use the app each hour of each day?,SELECT EXTRACT(HOUR FROM event_timestamp) as h...,"['event_timestamp', 'user_pseudo_id', 'event_n...",0.292959,148,0.065065,133
22,"On average, how many users have opened the app...","SELECT hour, AVG(cnt) as avg_cnt FROM\n (SELE...","['event_timestamp', 'user_pseudo_id', 'event_n...",0.292959,143,0.041250,131
30,How many users deleted the app last 30 days?,SELECT COUNT(DISTINCT user_pseudo_id) FROM tea...,"['user_pseudo_id', 'event_name', 'event_timest...",0.303645,116,0.061919,137
33,What is number of times users open X store las...,SELECT COUNT(event_name) FROM team_tds.tds_int...,"['event_timestamp', 'event_name']",0.336628,134,0.072696,146


In [7]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses, util
from datasets import load_dataset
from torch.utils.data import DataLoader

model = SentenceTransformer('all-MiniLM-L6-v2')
# dataset_id = "embedding-data/QQP_triplets"
dataset_id = "embedding-data/sentence-compression"
dataset = load_dataset(dataset_id)

Found cached dataset json (/Users/thanawatthongpia/.cache/huggingface/datasets/embedding-data___json/embedding-data--sentence-compression-d643585deb6e0073/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset['train']['set'][0]

In [ ]:
train_examples = []
n_examples = dataset['train'].num_rows // 20
train_data = dataset['train']['set']

for i in range(n_examples):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example[0], example[1]]))

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [ ]:
num_epochs = 2
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
len(train_dataloader)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

In [11]:
from Description_base_linking import SchemaLinking
import pandas as pd
import numpy as np
# import time

obj = SchemaLinking()
# obj.verbose = True

In [12]:
ques = 'what is the average number of event name occur from user in this month'
thres = 0.5
for c in obj.get_columns(ques ,thres):
    print(c,'\t', obj.q_c_similarity(ques,c))

event_month 	 0.5574487447738647


In [50]:
threshold = 0.15
compare_df = pd.read_csv('smilarity_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()
for i, q in enumerate(questions[:5]):
    print(q)
    expect_col = obj.columns_from_query(sql_queries[i])
    choose_col = obj.get_columns(q, threshold)

    print("EXPECT COLUMNS:\t\t", expect_col)
    # print('SELECTED COLUMNS:\t', choose_col)

    expect_col_vec = obj.choose_col_vec(expect_col)
    choose_col_vec = obj.choose_col_vec(choose_col)
    accuracy = obj.accuracy_of_vecs(choose_col_vec, expect_col_vec)

    not_selected_col_idx =  (choose_col_vec == 0) & (expect_col_vec ==1 ).astype(int)
    not_selected_col = [ obj.schema_columns[idx] for idx in np.where(not_selected_col_idx == 1)[0]]
    if len(not_selected_col):
        print("COL NOT SELECTED:\t",not_selected_col)

    print(accuracy)
    print()

How many daily active users each day?
EXPECT COLUMNS:		 ['engagement_time_msec', 'ga_session_id', 'event_date', 'user_pseudo_id']
1.0

How many monthly active users each month?
EXPECT COLUMNS:		 ['engagement_time_msec', 'ga_session_id', 'user_pseudo_id', 'event_month']
COL NOT SELECTED:	 ['ga_session_id']
0.75

What is the average number of daily active users last 7 days?
EXPECT COLUMNS:		 ['engagement_time_msec', 'ga_session_id', 'event_date', 'user_pseudo_id']
COL NOT SELECTED:	 ['ga_session_id']
0.75

What is the mean number of daily active users from last 30 days?
EXPECT COLUMNS:		 ['engagement_time_msec', 'ga_session_id', 'event_date', 'user_pseudo_id']
1.0

When was the last time each user was active?
EXPECT COLUMNS:		 ['ga_session_id', 'event_timestamp', 'engagement_time_msec', 'user_pseudo_id', 'event_date']
1.0

